In [1]:
import json
import openai
import os
import random

import numpy as np
import pandas as pd

Import openai api key from environment variable - here I have it stored as `LOCAL_OPENAI_API_KEY` but you can just paste yours in instead

In [2]:

openai.api_key = os.environ.get('LOCAL_OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
  """
  wrapper for the API to easily parse data
  """
  
  args = {
    "engine":"text-davinci-001", # using the original davinci
    "temperature":0, # 0 temperature means it's greedy and gives the same result every time (ish)
    "max_tokens":500, # 500 tokens should be enough
    "stop":"\n\n", # we'll use double newlines to separate the examples
  }
  
  args = {**args, **kwargs}
  
  r = openai.Completion.create(prompt=prompt, **args)["choices"][0]["text"].strip()
  return r


Great, the API key is loaded. Now we can start using the API.

In [4]:
query("q: what is 1+1?\na:")

'2'

This downloads the WIC dataset - you may need to install wget if you don't have it installed https://ports.macports.org/port/wget/

In [11]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-02-26 19:27:39--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 2606:50c0:8002::153, 2606:50c0:8000::153, 2606:50c0:8003::153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|2606:50c0:8002::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.05s   

2023-02-26 19:27:39 (5.17 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [12]:
import zipfile
with zipfile.ZipFile("WiC_dataset.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [4]:
train = pd.read_csv("train/train.data.txt", sep='\t', header=None)
train.columns = ["target", "pos", "position", "context-1", "context-2"]
train_gold = pd.read_csv("train/train.gold.txt", sep='\t', header=None)
train_gold.columns = ["label"]
train = pd.concat([train_gold,train], axis=1)


In [5]:
train[train.pos=="V"].tail(18)[:5]


,label,target,pos,position,context-1,context-2
5386,F,admit,V,3-1,The French doors admit onto the yard .,He admitted his errors .
5387,F,exhaust,V,0-4,Exhaust one 's savings .,This kind of work exhausts me .
5389,F,kill,V,0-2,Kill the engine .,She was killed in the collision of three cars .
5390,T,admit,V,1-1,To admit a serious thought into the mind .,She admitted us here .
5394,F,write,V,6-1,How many books did Georges Simenon write ?,Please write to me every week .


We can go and grab the last couple verbs of the train set to use as our few shots. I added some annotation for meaning

In [6]:

fewShotVerb = """ interface comparison{
    "Sense1": str, // the sense of the word in the first context
    "Sense2": str, // the sense of the word in the second context
    "areSimilar": bool, // whether the two senses are similar
}

determineWordSimilarSense(word, context1, context2) : comparison =>{
    return ai.compare(word, context1, context2) // return the comparison object
}

determineWordSimilarSense("The French doors admit onto the yard .", "He admitted his errors .", "admit")
>>> {
"Sense1": "In the first sentence, 'admit' means to provide passage.",
"Sense2": "In the second, 'admit' means to take responsibility.",
"Similar": true
}

determineWordSimilarSense("The company agrees to meet the cost of any repairs .", "Does this paper meet the requirements for the degree ?", "meet")
>>> {
"Sense1": "In the first sentence, 'meet' means to fulfill.",
"Sense2": "In the second, 'meet' means to fulfill.",
"Similar": true
}

determineWordSimilarSense("You anger too easily .", "He angers easily .", "anger")
>>> {
"Sense1": "In the first sentence, 'anger' means to enrage.",
"Sense2": "In the second, 'anger' means to enrage.",
"Similar": true
}

determineWordSimilarSense("Exhaust one 's savings .", "This kind of work exhausts me .", "exhaust")
>>> {
"Sense1": "In the first sentence, 'exhaust' means to empty.",
"Sense2": "In the second, 'exhaust' means to tire.",
"Similar": false
}

determineWordSimilarSense("Kill the engine .", "She was killed in the collision of three cars .", "kill")
>>> {
"Sense1": "In the first sentence, 'kill' means to turn off.",
"Sense2": "In the second, 'kill' refers to dying.",
"Similar": false
}

determineWordSimilarSense("To admit a serious thought into the mind .", "She admitted us here .", "admit")
>>> {
"Sense1": "In the first sentence, 'admit' means to allow in.",
"Sense2": "In the second, 'admit' means to allow in.",
"Similar": true
}

determineWordSimilarSense("How many books did Georges Simenon write ?", "Please write to me every week .", "write")
>>> {
"Sense1": "In the first sentence, 'write' means to publish.",
"Sense2": "In the second, 'write' means to communicate with.",
"Similar": false
}

determineWordSimilarSense("This skill will enable you to find a job on Wall Street .", "The rope enables you to secure yourself when you climb the mountain .", "enable")
>>> {
"Sense1": "In the first sentence, 'enable' means to give an ability.",
"Sense2": "In the second, 'enable' means to give an ability.",
"Similar": true
}

determineWordSimilarSense("The man must answer to his employer for the money entrusted to his care .", "She must answer for her actions .", "answer")
>>> {
"Sense1": "In the first sentence, 'answer' means to take responsibility.",
"Sense2": "In the second, ''answer' means to take responsibility. In the second, 'answer' means to take responsibility.",
"Similar": true
}

determineWordSimilarSense("Hit the bottle .", "He hit a home run .", "hit")
>>> {
"Sense1": "In the first sentence, 'hit' means to use.",
"Sense2": "In the second, 'hit' means to strike.",
"Similar": false
}

"""

Likewise we can do it with nouns

In [7]:
fewShotNoun = """determineWordSimilarSense("A history of France .", "A critical time in the school 's history .", "history")
>>> {
"Sense1": "In the first sentence, 'history' refers to a record.",
"Sense2": "In the second sentence, 'history' means past.",
"Similar": false
}

determineWordSimilarSense("I do it for the fun of it .", "He is fun to have around .", "fun")
>>> {
"Sense1": "In the first sentence, 'fun' means having pleasure.",
"Sense2": "In the second sentence, 'fun' also means pleasurable.",
"Similar": true
}

determineWordSimilarSense("The rate of production at the factory is skyrocketing .", "He works at a great rate .", "rate")
>>> {
"Sense1": "In the first sentence, 'rate' refers to speed.",
"Sense2": "In the second sentence, 'rate' also means speed.",
"Similar": true
}

determineWordSimilarSense("Get to the point .", "At that point I had to leave .", "point")
>>> {
"Sense1": "In the first sentence, 'point' refers to a topic.",
"Sense2": "In the second sentence, 'point' means time.",
"Similar": false
}

determineWordSimilarSense("Kronas kurss — the exchange rate of the krona .", "Grāmata maksā piecas kronas — the book costs five krona .", "krona")
>>> {
"Sense1": "In the first sentence, 'krona' means money.",
"Sense2": "In the second sentence, 'krona' also means money.",
"Similar": true
}

determineWordSimilarSense("Armored from head to foot .", "The swiftest of foot .", "foot")
>>> {
"Sense1": "In the first sentence, 'foot' refers to a body part.",
"Sense2": "In the second sentence, 'foot' means speed.",
"Similar": false
}

determineWordSimilarSense("A patch of clouds .", "Patches of thin ice .", "patch")
>>> {
"Sense1": "In the first sentence, 'patch' means cluster.",
"Sense2": "In the second sentence, 'patch' also means cluster.",
"Similar": true
}

determineWordSimilarSense("The misery and wretchedness of those slums is intolerable .", "She was exhausted by her misery and grief .", "misery")
>>> {
"Sense1": "In the first sentence, 'misery' means a state of unhappiness.",
"Sense2": "In the second sentence, 'misery' refers to a feeling of unhappiness.",
"Similar": false
}

determineWordSimilarSense("Women carrying home shopping did n't give me a second glance .", "On Saturdays we usually do the shopping .", "shopping")
>>> {
"Sense1": "In the first sentence, 'shopping' refers to purchases.",
"Sense2": "In the second it means buying things.",
"Similar": false
}

determineWordSimilarSense("While being impulsive can be great for artists , it is not a desirable quality for engineers .", "Security , stability , and efficiency are good qualities of an operating system .", "quality")
>>> {
"Sense1": "In the first sentence, 'quality' means attribute",
"Sense2": "In the second it means attribute",
"Similar": true
}

"""

In [8]:
def testRow(row):
  pos = row[1]["pos"]
  examples = fewShotNoun if pos == "N" else fewShotVerb

  context = """determineWordSimilarSense("{}", "{}", "{}")
>>>""".format(row[1]["context-1"], row[1]["context-2"], row[1]["target"])
  res = query(examples + context)
  try:
    parsed = json.loads(res)
  except Exception as e:
    # print('error parsing: ', e, res)
    # check if there's a true in the last 100 characters
    parsed = {
      "Context": res,
      "Similar": "true" in res[-100:]
    }

  return parsed


In [9]:
scores = {'V': 0, 'N': 0}
attempted = {'V': 0, 'N': 0}
for row in train.head(10).iterrows():
    print('scores', scores)
    print('attempted', attempted)
    actual = row[1]["label"]
    output = testRow(row)
    print('output:', output)
    pos = row[1]["pos"]
    attempted[pos] += 1

    if actual == "T":
        if output['Similar']:
            scores[pos] += 1
        
    if actual == "F":
        if not output['Similar']:
            scores[pos] += 1



scores {'V': 0, 'N': 0}
attempted {'V': 0, 'N': 0}
output: {'Context': '{\n"Sense1": "In the first sentence, \'carry\' means to transport.",\n"Sense2": "In the second, \'carry\' means to transmit.",\n"Similar": true', 'Similar': True}
scores {'V': 0, 'N': 0}
attempted {'V': 1, 'N': 0}
output: {'Context': '{\n"Sense1": "In the first sentence, \'go\' means to send.",\n"Sense2": "In the second, \'go\' means to fit.",\n"Similar": false', 'Similar': False}
scores {'V': 1, 'N': 0}
attempted {'V': 2, 'N': 0}
output: {'Context': '{\n"Sense1": "In the first sentence, \'break\' means to divide.",\n"Sense2": "In the second, \'break\' means to destroy.",\n"Similar": true', 'Similar': True}
scores {'V': 1, 'N': 0}
attempted {'V': 3, 'N': 0}
output: {'Context': '', 'Similar': False}
scores {'V': 1, 'N': 0}
attempted {'V': 3, 'N': 1}
output: {'Context': '', 'Similar': False}
scores {'V': 1, 'N': 1}
attempted {'V': 3, 'N': 2}
output: {'Context': '{\n"Sense1": "In the first sentence, \'set\' means to p

In [10]:
scores, attempted

({'V': 3, 'N': 1}, {'V': 7, 'N': 3})

In [11]:
dev = pd.read_csv("dev/dev.data.txt", sep='\t', header=None)
dev.columns = ["target", "pos", "position", "context-1", "context-2"]
dev_gold = pd.read_csv("dev/dev.gold.txt", sep='\t', header=None)
dev_gold.columns = ["label"]
dev = pd.concat([dev_gold,dev], axis=1)


In [12]:
devResults = {}
complete = 0
correct = 0

In [13]:

for row in dev.iterrows():

    if row[0] in devResults:
        continue

    q1 = row[1]["context-1"]
    q2 = row[1]["context-2"]
    target = row[1]["target"]
    actual = row[1]["label"]
    
    pos = row[1]["pos"]

    output = testRow(row)
    
    myResults = {}
    myResults["q1"] = q1
    myResults["q2"] = q2

    myResults["pos"] = row[1]["pos"]

    myResults["target"] = target

    myResults["output"] = output

    myResults["actual"] = actual
    devResults[row[0]] = myResults
    complete +=1
    if actual == "T":
        if output['Similar']:
            correct += 1
    if actual == "F":
        if not output['Similar']:
            correct += 1

    pct = correct/complete
    pct = round(pct, 2)

    print ("Score: {}, Complete: {} Correct: {} Wrong: {}".format(pct, complete, correct, complete-correct))
    with open('text-davinci-001-code.json', 'w') as f:
        json.dump(devResults, f)


Score: 1.0, Complete: 1 Correct: 1 Wrong: 0
Score: 0.5, Complete: 2 Correct: 1 Wrong: 1
Score: 0.67, Complete: 3 Correct: 2 Wrong: 1
Score: 0.75, Complete: 4 Correct: 3 Wrong: 1
Score: 0.8, Complete: 5 Correct: 4 Wrong: 1
Score: 0.83, Complete: 6 Correct: 5 Wrong: 1
Score: 0.86, Complete: 7 Correct: 6 Wrong: 1
Score: 0.88, Complete: 8 Correct: 7 Wrong: 1
Score: 0.78, Complete: 9 Correct: 7 Wrong: 2
Score: 0.8, Complete: 10 Correct: 8 Wrong: 2
Score: 0.82, Complete: 11 Correct: 9 Wrong: 2
Score: 0.83, Complete: 12 Correct: 10 Wrong: 2
Score: 0.77, Complete: 13 Correct: 10 Wrong: 3
Score: 0.79, Complete: 14 Correct: 11 Wrong: 3
Score: 0.8, Complete: 15 Correct: 12 Wrong: 3
Score: 0.81, Complete: 16 Correct: 13 Wrong: 3
Score: 0.82, Complete: 17 Correct: 14 Wrong: 3
Score: 0.83, Complete: 18 Correct: 15 Wrong: 3
Score: 0.84, Complete: 19 Correct: 16 Wrong: 3
Score: 0.85, Complete: 20 Correct: 17 Wrong: 3
Score: 0.81, Complete: 21 Correct: 17 Wrong: 4
Score: 0.77, Complete: 22 Correct: 17 

In [ ]:
with open('text-davinci-001-code.json', 'w') as f:
    json.dump(devResults, f)
